In [2]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/home/vimalkumar/Documents/Classification/Train'
valid_path = '/home/vimalkumar/Documents/Classification/Test'

In [4]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

2021-11-17 17:17:08.939150: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-17 17:17:08.939180: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-17 17:17:08.939202: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (IG154): /proc/driver/nvidia/version does not exist
2021-11-17 17:17:08.939399: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [6]:
# useful for getting number of output classes
folders = glob('/home/vimalkumar/Documents/Classification/Train/*')
folders

['/home/vimalkumar/Documents/Classification/Train/Marksheet',
 '/home/vimalkumar/Documents/Classification/Train/License',
 '/home/vimalkumar/Documents/Classification/Train/Degree_certificates',
 '/home/vimalkumar/Documents/Classification/Train/Invoice',
 '/home/vimalkumar/Documents/Classification/Train/Discharge']

In [7]:
# our layers - you can add more if you want
x = Flatten()(inception.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [9]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [10]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/home/vimalkumar/Documents/Classification/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 2,
                                                 class_mode = 'categorical')

Found 552 images belonging to 5 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/home/vimalkumar/Documents/Classification/Test',
                                            target_size = (224, 224),
                                            batch_size = 2,
                                            class_mode = 'categorical')

Found 60 images belonging to 5 classes.


In [14]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=1,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-11-17 17:17:43.873795: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


 73/276 [======>.......................] - ETA: 1:01 - loss: 6.8788 - accuracy: 0.5616

/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (167630848 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


106/276 [==========>...................] - ETA: 52s - loss: 5.2228 - accuracy: 0.6604

/home/vimalkumar/Documents/Python_new_1/cde_venv/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (171400164 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


276/276 [==============================] - 96s 334ms/step - loss: 3.2666 - accuracy: 0.7880 - val_loss: 1.2710 - val_accuracy: 0.8500


In [ ]:
#y_pred = model.predict(test_set)


In [ ]:
#y_pred

In [15]:
import numpy as np
#y_pred = np.argmax(y_pred, axis=1)

In [16]:
#y_pred


In [17]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [18]:
from tensorflow.keras.models import load_model

model.save('model_inception.h5')

In [46]:
model=load_model('model_inception.h5')


In [47]:
im = '/home/vimalkumar/Documents/Classification/Test/Degree_certificates/degree-certificate-printing-services-500x500.jpeg'
img=image.load_img(im,
                   target_size=(224,224))


In [48]:
x=image.img_to_array(img)
#x

In [49]:
x=x/255

In [50]:
x=np.expand_dims(x,axis=0)
#print(x)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [51]:
model.predict(img_data)

array([[1.3389528e-05, 5.0814116e-01, 4.9183780e-01, 7.6640245e-06,
        1.0671311e-08]], dtype=float32)

In [52]:
a=np.argmax(model.predict(img_data),axis=1)
a

array([1])

In [53]:
import os
keys = []
values = []
for key in range(len(folders)):
    #print(key)
    keys.append(key)
for path in os.listdir('/home/vimalkumar/Documents/Classification/Train'):
    #print(path)
    values.append(path)
dic = dict(list(zip(keys,sorted(values))))
for val in a:
    #print(v)
    
    print(dic[val])

Discharge
